In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import os
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
from sklearn.metrics import r2_score
from scipy.interpolate import interp1d
import tensorflow_datasets as tfds
from collections import Counter
import scipy.ndimage
from tensorflow.keras.callbacks import ModelCheckpoint
import keras
import gc
import random
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import tarfile
import urllib.request
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch.nn.functional as F
from scipy.stats import gaussian_kde
from numba import jit
from torchvision.datasets import Places365
from tqdm import tqdm  # 導入進度條庫
import json


os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
torch.backends.cudnn.benchmark = True
limit = 1000

# bzq modifying
len_x_target = 20
len_y_target = 20
stride_x_target = 10
stride_y_target = 10

# mean, std proportion
alpha = 0.5

bins_size = 30  # 統計採樣數
poly_degree = bins_size - 1
window_size = 1

#target image preprocessing
angle = 0
pixels = 0

In [ ]:
# 設置設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),  # Convert to tensor (values in range 0–1)
    transforms.Lambda(lambda x: x * 255.0),  # Scale to range 0–255
    transforms.Lambda(lambda x: x[[2, 1, 0], ...]),  # Convert RGB to BGR
    transforms.Normalize(mean=[103.939, 116.779, 123.68], std=[1.0, 1.0, 1.0])  # Subtract mean
])
# 載入 Places365 數據集
dataset = Places365(
    root="/home/a/bzq_on_confidence/Confidence2022/notebook/confident/grocery/Places365",
    split="train-standard",  # 指定數據集分割，例如 "train-standard", "val"
    small=True,             # 是否使用小尺寸圖像（256x256）
    transform=transform,     # 圖像增強
    target_transform=None,   # 標籤增強
    download=True            # 自動下載數據集
)

In [ ]:

def adjust_learning_rate(optimizer, epoch, initial_lr):
    if epoch <= 5:
        lr = initial_lr * epoch / 5
    elif epoch > 160:
        lr = initial_lr * 0.01
    elif epoch > 75:
        lr = initial_lr * 0.1
    else:
        lr = initial_lr

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Define the VGG16 model
model = models.vgg16(weights=False)  # Initialize without pre-trained weights
model = model.to(device)
num_features = model.classifier[6].in_features  # Get the input features of the last FC layer
model.classifier[6] = torch.nn.Linear(num_features, 365)  # Modify the last layer for 365 classes

model.features = model.features.to(device)
model.classifier = model.classifier.to(device)


# 定義損失函數和優化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
num_epochs = 200

if os.path.exists(f"vgg16_places365.pt"):
    # 加載保存的權重
    checkpoint = torch.load("vgg16_places365.pt")

    # Mapping Caffe keys to PyTorch keys
    mapped_state_dict = {
        # Conv1 Layers
        "features.0.weight": checkpoint["conv1_1.weight"],
        "features.0.bias": checkpoint["conv1_1.bias"],
        "features.2.weight": checkpoint["conv1_2.weight"],
        "features.2.bias": checkpoint["conv1_2.bias"],
        
        # Conv2 Layers
        "features.5.weight": checkpoint["conv2_1.weight"],
        "features.5.bias": checkpoint["conv2_1.bias"],
        "features.7.weight": checkpoint["conv2_2.weight"],
        "features.7.bias": checkpoint["conv2_2.bias"],
        
        # Conv3 Layers
        "features.10.weight": checkpoint["conv3_1.weight"],
        "features.10.bias": checkpoint["conv3_1.bias"],
        "features.12.weight": checkpoint["conv3_2.weight"],
        "features.12.bias": checkpoint["conv3_2.bias"],
        "features.14.weight": checkpoint["conv3_3.weight"],
        "features.14.bias": checkpoint["conv3_3.bias"],
        
        # Conv4 Layers
        "features.17.weight": checkpoint["conv4_1.weight"],
        "features.17.bias": checkpoint["conv4_1.bias"],
        "features.19.weight": checkpoint["conv4_2.weight"],
        "features.19.bias": checkpoint["conv4_2.bias"],
        "features.21.weight": checkpoint["conv4_3.weight"],
        "features.21.bias": checkpoint["conv4_3.bias"],
        
        # Conv5 Layers
        "features.24.weight": checkpoint["conv5_1.weight"],
        "features.24.bias": checkpoint["conv5_1.bias"],
        "features.26.weight": checkpoint["conv5_2.weight"],
        "features.26.bias": checkpoint["conv5_2.bias"],
        "features.28.weight": checkpoint["conv5_3.weight"],
        "features.28.bias": checkpoint["conv5_3.bias"],
        
        # Fully Connected Layers
        "classifier.0.weight": checkpoint["fc6.weight"],
        "classifier.0.bias": checkpoint["fc6.bias"],
        "classifier.3.weight": checkpoint["fc7.weight"],
        "classifier.3.bias": checkpoint["fc7.bias"],
        "classifier.6.weight": checkpoint["fc8a.weight"],
        "classifier.6.bias": checkpoint["fc8a.bias"],
    }

    # Load the updated state dict into the model
    model.load_state_dict(mapped_state_dict)
    print(model.features)
    # 切換模型到評估模式
    model.eval()
else:
    # 獲取數據
    image, target = dataset[0]
    print(f"Image size: {image.size()}, Target: {target}")
    
    # 計算每個類別的樣本數
    targets = [sample[1] for sample in dataset]  # 假設 dataset 返回 (data, label)
    class_counts = torch.bincount(torch.tensor(targets))
    class_weights = 1.0 / class_counts.float()
    weights = torch.tensor([class_weights[label] for label in targets])

    # 創建 WeightedRandomSampler
    sampler = torch.utils.data.WeightedRandomSampler(
        weights=weights, 
        num_samples=len(weights), 
        replacement=True
    )

    train_loader = DataLoader(dataset, batch_size=80, sampler=sampler, shuffle=False, num_workers=6, pin_memory=True)
    val_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=6, pin_memory=True)
    print("ready")
    
    scaler = torch.amp.GradScaler()

    for epoch in range(num_epochs):
        adjust_learning_rate(optimizer, epoch, 0.1)

        model.train()  # 設定模型為訓練模式
        running_loss = 0.0
        batch_count = 0

        # 使用 tqdm 包裹 DataLoader，顯示進度條
        with tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch") as progress_bar:
            for images, labels in progress_bar:
                images, labels = images.to(device), labels.to(device)

                # 混合精度前向傳播
                with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                # 縮放損失並進行反向傳播
                scaler.scale(loss).backward()

                # 更新參數
                scaler.step(optimizer)
                scaler.update()

                # 清零梯度
                optimizer.zero_grad()

                running_loss += loss.item()
                batch_count += 1
                # 在進度條中顯示當前損失
                progress_bar.set_postfix(loss=f"{running_loss / batch_count:.4f}")

        # 評估準確度
        model.eval()  # 設定模型為評估模式
        correct = 0
        total = 0
        
        with torch.no_grad():  # 停用梯度計算
            for images, labels in val_loader:  
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)  # 獲取預測標籤
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch + 1}, Accuracy: {accuracy:.2f}%")

        # 保存模型權重
        torch.save(model.state_dict(), f"Places365_vgg16_epoch_{epoch + 1}.pth")
        print(f"Model saved for epoch {epoch + 1}")

    print("Training complete!")


In [ ]:
# 得到了bzq 正確的函數，拿來做imagenet 正確的預測

def single_data_bzq_mask_preprocessing_imagenet(original_data, start_x, start_y, len_x, len_y, magnification):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data)
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] *= magnification
    #new_data[start_y:start_y + len_y, start_x:start_x + len_x, :] *= magnification
    return new_data


#print(random_num_for_bzq_mask_imagenet)

def single_data_bzq_mask_preprocessing_imagenet_random_global(original_data, start_x, start_y, len_x, len_y, random_num_for_bzq_mask_imagenet):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data.cpu().numpy())
    new_data = new_data.squeeze(0)
    random_num_for_bzq_mask_imagenet = random_num_for_bzq_mask_imagenet[:, :len_y, :len_x] 
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] = random_num_for_bzq_mask_imagenet
    return new_data


bzq = []
correct_predictions_imagenet = []
incorrect_predictions_imagenet = []
bzq_imagenet = []

nll_bzq = []

#bzq = 0的時候，提取mask之下的softmax
correct_predictions_bzq_zero_softmax_mean = []
correct_predictions_bzq_zero_softmax_std = []
incorrect_predictions_bzq_zero_softmax_mean = []
incorrect_predictions_bzq_zero_softmax_std = []


In [ ]:
#--- Corruptions ---
import skimage as sk
from skimage.filters import gaussian
from io import BytesIO
from wand.image import Image as WandImage
from wand.api import library as wandlibrary
import wand.color as WandColor
import ctypes
from PIL import Image as PILImage
import cv2
from scipy.ndimage import zoom as scizoom
from scipy.ndimage.interpolation import map_coordinates
import warnings

warnings.simplefilter("ignore", UserWarning)


def auc(errs):  # area under the alteration error curve
    area = 0
    for i in range(1, len(errs)):
        area += (errs[i] + errs[i - 1]) / 2
    area /= len(errs) - 1
    return area


def disk(radius, alias_blur=0.1, dtype=np.float32):
    if radius <= 8:
        L = np.arange(-8, 8 + 1)
        ksize = (3, 3)
    else:
        L = np.arange(-radius, radius + 1)
        ksize = (5, 5)
    X, Y = np.meshgrid(L, L)
    aliased_disk = np.array((X ** 2 + Y ** 2) <= radius ** 2, dtype=dtype)
    aliased_disk /= np.sum(aliased_disk)

    # supersample disk to antialias
    return cv2.GaussianBlur(aliased_disk, ksize=ksize, sigmaX=alias_blur)


# Tell Python about the C method
wandlibrary.MagickMotionBlurImage.argtypes = (ctypes.c_void_p,  # wand
                                              ctypes.c_double,  # radius
                                              ctypes.c_double,  # sigma
                                              ctypes.c_double)  # angle


# Extend wand.image.Image class to include method signature
class MotionImage(WandImage):
    def motion_blur(self, radius=0.0, sigma=0.0, angle=0.0):
        wandlibrary.MagickMotionBlurImage(self.wand, radius, sigma, angle)


# modification of https://github.com/FLHerne/mapgen/blob/master/diamondsquare.py
def plasma_fractal(mapsize=256, wibbledecay=3):
    """
    Generate a heightmap using diamond-square algorithm.
    Return square 2d array, side length 'mapsize', of floats in range 0-255.
    'mapsize' must be a power of two.
    """
    assert (mapsize & (mapsize - 1) == 0)
    maparray = np.empty((mapsize, mapsize), dtype=np.float_)
    maparray[0, 0] = 0
    stepsize = mapsize
    wibble = 100

    def wibbledmean(array):
        return array / 4 + wibble * np.random.uniform(-wibble, wibble, array.shape)

    def fillsquares():
        """For each square of points stepsize apart,
           calculate middle value as mean of points + wibble"""
        cornerref = maparray[0:mapsize:stepsize, 0:mapsize:stepsize]
        squareaccum = cornerref + np.roll(cornerref, shift=-1, axis=0)
        squareaccum += np.roll(squareaccum, shift=-1, axis=1)
        maparray[stepsize // 2:mapsize:stepsize,
        stepsize // 2:mapsize:stepsize] = wibbledmean(squareaccum)

    def filldiamonds():
        """For each diamond of points stepsize apart,
           calculate middle value as mean of points + wibble"""
        mapsize = maparray.shape[0]
        drgrid = maparray[stepsize // 2:mapsize:stepsize, stepsize // 2:mapsize:stepsize]
        ulgrid = maparray[0:mapsize:stepsize, 0:mapsize:stepsize]
        ldrsum = drgrid + np.roll(drgrid, 1, axis=0)
        lulsum = ulgrid + np.roll(ulgrid, -1, axis=1)
        ltsum = ldrsum + lulsum
        maparray[0:mapsize:stepsize, stepsize // 2:mapsize:stepsize] = wibbledmean(ltsum)
        tdrsum = drgrid + np.roll(drgrid, 1, axis=1)
        tulsum = ulgrid + np.roll(ulgrid, -1, axis=0)
        ttsum = tdrsum + tulsum
        maparray[stepsize // 2:mapsize:stepsize, 0:mapsize:stepsize] = wibbledmean(ttsum)

    while stepsize >= 2:
        fillsquares()
        filldiamonds()
        stepsize //= 2
        wibble /= wibbledecay

    maparray -= maparray.min()
    return maparray / maparray.max()


def clipped_zoom(img, zoom_factor):
    h = img.shape[0]
    # ceil crop height(= crop width)
    ch = int(np.ceil(h / zoom_factor))

    top = (h - ch) // 2
    img = scizoom(img[top:top + ch, top:top + ch], (zoom_factor, zoom_factor, 1), order=1)
    # trim off any extra pixels
    trim_top = (img.shape[0] - h) // 2

    return img[trim_top:trim_top + h, trim_top:trim_top + h]


# /////////////// End Distortion Helpers ///////////////


# /////////////// Distortions ///////////////

def gaussian_noise(x, severity=1):
    c = [.08, .12, 0.18, 0.26, 0.38][severity - 1]

    x = np.array(x) / 255.
    return np.clip(x + np.random.normal(size=x.shape, scale=c), 0, 1) * 255


def shot_noise(x, severity=1):
    c = [60, 25, 12, 5, 3][severity - 1]

    x = np.array(x) / 255.
    return np.clip(np.random.poisson(x * c) / c, 0, 1) * 255



def impulse_noise(x, severity=1):
    c = [.03, .06, .09, 0.17, 0.27][severity - 1]

    x = sk.util.random_noise(np.array(x) / 255., mode='s&p', amount=c)
    return np.clip(x, 0, 1) * 255


def speckle_noise(x, severity=1):
    c = [.15, .2, 0.35, 0.45, 0.6][severity - 1]

    x = np.array(x) / 255.
    return np.clip(x + x * np.random.normal(size=x.shape, scale=c), 0, 1) * 255


'''def fgsm(x, source_net, severity=1):
    c = [8, 16, 32, 64, 128][severity - 1]

    x = V(x, requires_grad=True)
    logits = source_net(x)
    source_net.zero_grad()
    loss = F.cross_entropy(logits, V(logits.data.max(1)[1].squeeze_()), size_average=False)
    loss.backward()

    return standardize(torch.clamp(unstandardize(x.data) + c / 255. * unstandardize(torch.sign(x.grad.data)), 0, 1))'''


def gaussian_blur(x, severity=1):
    c = [1, 2, 3, 4, 6][severity - 1]

    x = gaussian(np.array(x) / 255., sigma=c, channel_axis=-1)
    return np.clip(x, 0, 1) * 255


def glass_blur(x, severity=1):
    # sigma, max_delta, iterations
    c = [(0.7, 1, 2), (0.9, 2, 1), (1, 2, 3), (1.1, 3, 2), (1.5, 4, 2)][severity - 1]

    x = np.uint8(gaussian(np.array(x) / 255., sigma=c[0], channel_axis=-1) * 255)

    # locally shuffle pixels
    for i in range(c[2]):
        for h in range(224 - c[1], c[1], -1):
            for w in range(224 - c[1], c[1], -1):
                dx, dy = np.random.randint(-c[1], c[1], size=(2,))
                h_prime, w_prime = h + dy, w + dx
                # swap
                x[h, w], x[h_prime, w_prime] = x[h_prime, w_prime], x[h, w]

    return np.clip(gaussian(x / 255., sigma=c[0], channel_axis=-1), 0, 1) * 255


def defocus_blur(x, severity=1):
    c = [(3, 0.1), (4, 0.5), (6, 0.5), (8, 0.5), (10, 0.5)][severity - 1]

    x = np.array(x) / 255.
    kernel = disk(radius=c[0], alias_blur=c[1])

    channels = []
    for d in range(3):
        channels.append(cv2.filter2D(x[:, :, d], -1, kernel))
    channels = np.array(channels).transpose((1, 2, 0))  # 3x224x224 -> 224x224x3

    return np.clip(channels, 0, 1) * 255


def motion_blur(x, severity=1):
    c = [(10, 3), (15, 5), (15, 8), (15, 12), (20, 15)][severity - 1]

    output = BytesIO()
    x.save(output, format='PNG')
    x = MotionImage(blob=output.getvalue())

    x.motion_blur(radius=c[0], sigma=c[1], angle=np.random.uniform(-45, 45))

    x = cv2.imdecode(np.fromstring(x.make_blob(), np.uint8),
                     cv2.IMREAD_UNCHANGED)

    if x.shape != (224, 224):
        return np.clip(x[..., [2, 1, 0]], 0, 255)  # BGR to RGB
    else:  # greyscale to RGB
        return np.clip(np.array([x, x, x]).transpose((1, 2, 0)), 0, 255)


def zoom_blur(x, severity=1):
    c = [np.arange(1, 1.11, 0.01),
         np.arange(1, 1.16, 0.01),
         np.arange(1, 1.21, 0.02),
         np.arange(1, 1.26, 0.02),
         np.arange(1, 1.31, 0.03)][severity - 1]

    x = (np.array(x) / 255.).astype(np.float32)
    out = np.zeros_like(x)
    for zoom_factor in c:
        out += clipped_zoom(x, zoom_factor)

    x = (x + out) / (len(c) + 1)
    return np.clip(x, 0, 1) * 255

def fog(x, severity=1):
    c = [(1.5, 2), (2, 2), (2.5, 1.7), (2.5, 1.5), (3, 1.4)][severity - 1]

    x = np.array(x) / 255.
    max_val = x.max()
    x += c[0] * plasma_fractal(wibbledecay=c[1])[:224, :224][..., np.newaxis]
    return np.clip(x * max_val / (max_val + c[0]), 0, 1) * 255


'''def frost(x, severity=1):
    c = [(1, 0.4),
         (0.8, 0.6),
         (0.7, 0.7),
         (0.65, 0.7),
         (0.6, 0.75)][severity - 1]
    idx = np.random.randint(5)
    filename = ['./frost1.png', './frost2.png', './frost3.png', './frost4.jpg', './frost5.jpg', './frost6.jpg'][idx]
    frost = cv2.imread(filename)
    # randomly crop and convert to rgb
    x_start, y_start = np.random.randint(0, frost.shape[0] - 224), np.random.randint(0, frost.shape[1] - 224)
    frost = frost[x_start:x_start + 224, y_start:y_start + 224][..., [2, 1, 0]]

    return np.clip(c[0] * np.array(x) + c[1] * frost, 0, 255)'''


'''def snow(x, severity=1):
    c = [(0.1, 0.3, 3, 0.5, 10, 4, 0.8),
         (0.2, 0.3, 2, 0.5, 12, 4, 0.7),
         (0.55, 0.3, 4, 0.9, 12, 8, 0.7),
         (0.55, 0.3, 4.5, 0.85, 12, 8, 0.65),
         (0.55, 0.3, 2.5, 0.85, 12, 12, 0.55)][severity - 1]

    x = np.array(x, dtype=np.float32) / 255.
    snow_layer = np.random.normal(size=x.shape[:2], loc=c[0], scale=c[1])  # [:2] for monochrome

    snow_layer = clipped_zoom(snow_layer[..., np.newaxis], c[2])
    snow_layer[snow_layer < c[3]] = 0

    snow_layer = PILImage.fromarray((np.clip(snow_layer.squeeze(), 0, 1) * 255).astype(np.uint8), mode='L')
    output = BytesIO()
    snow_layer.save(output, format='PNG')
    snow_layer = MotionImage(blob=output.getvalue())

    snow_layer.motion_blur(radius=c[4], sigma=c[5], angle=np.random.uniform(-135, -45))

    snow_layer = cv2.imdecode(np.fromstring(snow_layer.make_blob(), np.uint8),
                              cv2.IMREAD_UNCHANGED) / 255.
    snow_layer = snow_layer[..., np.newaxis]

    x = c[6] * x + (1 - c[6]) * np.maximum(x, cv2.cvtColor(x, cv2.COLOR_RGB2GRAY).reshape(224, 224, 1) * 1.5 + 0.5)
    return np.clip(x + snow_layer + np.rot90(snow_layer, k=2), 0, 1) * 255'''


def spatter(x, severity=1):
    c = [(0.65, 0.3, 4, 0.69, 0.6, 0),
         (0.65, 0.3, 3, 0.68, 0.6, 0),
         (0.65, 0.3, 2, 0.68, 0.5, 0),
         (0.65, 0.3, 1, 0.65, 1.5, 1),
         (0.67, 0.4, 1, 0.65, 1.5, 1)][severity - 1]
    x = np.array(x, dtype=np.float32) / 255.

    liquid_layer = np.random.normal(size=x.shape[:2], loc=c[0], scale=c[1])

    liquid_layer = gaussian(liquid_layer, sigma=c[2])
    liquid_layer[liquid_layer < c[3]] = 0
    if c[5] == 0:
        liquid_layer = (liquid_layer * 255).astype(np.uint8)
        dist = 255 - cv2.Canny(liquid_layer, 50, 150)
        dist = cv2.distanceTransform(dist, cv2.DIST_L2, 5)
        _, dist = cv2.threshold(dist, 20, 20, cv2.THRESH_TRUNC)
        dist = cv2.blur(dist, (3, 3)).astype(np.uint8)
        dist = cv2.equalizeHist(dist)
        #     ker = np.array([[-1,-2,-3],[-2,0,0],[-3,0,1]], dtype=np.float32)
        #     ker -= np.mean(ker)
        ker = np.array([[-2, -1, 0], [-1, 1, 1], [0, 1, 2]])
        dist = cv2.filter2D(dist, cv2.CV_8U, ker)
        dist = cv2.blur(dist, (3, 3)).astype(np.float32)

        m = cv2.cvtColor(liquid_layer * dist, cv2.COLOR_GRAY2BGRA)
        m /= np.max(m, axis=(0, 1))
        m *= c[4]

        # water is pale turqouise
        color = np.concatenate((175 / 255. * np.ones_like(m[..., :1]),
                                238 / 255. * np.ones_like(m[..., :1]),
                                238 / 255. * np.ones_like(m[..., :1])), axis=2)

        color = cv2.cvtColor(color, cv2.COLOR_BGR2BGRA)
        x = cv2.cvtColor(x, cv2.COLOR_BGR2BGRA)

        return cv2.cvtColor(np.clip(x + m * color, 0, 1), cv2.COLOR_BGRA2BGR) * 255
    else:
        m = np.where(liquid_layer > c[3], 1, 0)
        m = gaussian(m.astype(np.float32), sigma=c[4])
        m[m < 0.8] = 0
        #         m = np.abs(m) ** (1/c[4])

        # mud brown
        color = np.concatenate((63 / 255. * np.ones_like(x[..., :1]),
                                42 / 255. * np.ones_like(x[..., :1]),
                                20 / 255. * np.ones_like(x[..., :1])), axis=2)

        color *= m[..., np.newaxis]
        x *= (1 - m[..., np.newaxis])

        return np.clip(x + color, 0, 1) * 255


def contrast(x, severity=1):
    c = [0.4, .3, .2, .1, .05][severity - 1]

    x = np.array(x) / 255.
    means = np.mean(x, axis=(0, 1), keepdims=True)
    return np.clip((x - means) * c + means, 0, 1) * 255


def brightness(x, severity=1):
    c = [.1, .2, .3, .4, .5][severity - 1]

    x = np.array(x) / 255.
    x = sk.color.rgb2hsv(x)
    x[:, :, 2] = np.clip(x[:, :, 2] + c, 0, 1)
    x = sk.color.hsv2rgb(x)

    return np.clip(x, 0, 1) * 255


def saturate(x, severity=1):
    c = [(0.3, 0), (0.1, 0), (2, 0), (5, 0.1), (20, 0.2)][severity - 1]

    x = np.array(x) / 255.
    x = sk.color.rgb2hsv(x)
    x[:, :, 1] = np.clip(x[:, :, 1] * c[0] + c[1], 0, 1)
    x = sk.color.hsv2rgb(x)

    return np.clip(x, 0, 1) * 255


def jpeg_compression(x, severity=1):
    c = [25, 18, 15, 10, 7][severity - 1]

    output = BytesIO()
    x.save(output, 'JPEG', quality=c)
    x = PILImage.open(output)

    return x


'''def pixelate(x, severity=1):
    x = PILImage.fromarray(x)
    c = [0.6, 0.5, 0.4, 0.3, 0.25][severity - 1]

    x = x.resize((int(224 * c), int(224 * c)), PILImage.BOX)
    x = x.resize((224, 224), PILImage.BOX)

    x = np.array(x)
    return x'''
def pixelate(x, severity=1):
    """
    Pixelates an image based on severity level.

    Parameters:
    x (numpy.ndarray): Input image.
    severity (int): Severity level (1 to 5).

    Returns:
    numpy.ndarray: Pixelated image.
    """

    # 確保輸入是 NumPy 陣列
    if not isinstance(x, np.ndarray):
        raise TypeError("Input 'x' must be a NumPy array.")
    
    # 確保數據類型為 uint8
    if x.dtype != np.uint8:
        x = (x * 255).astype(np.uint8)

    # 單通道處理（如灰階），確保轉換為 RGB
    if x.ndim == 3 and x.shape[-1] == 1:
        x = np.squeeze(x, axis=-1)
        x = np.stack([x] * 3, axis=-1)  # 灰階轉換為三通道 RGB

    # PIL 圖片轉換
    x = PILImage.fromarray(x)
    
    # 確保 severity 在合理範圍內
    severity = int(np.clip(severity, 1, 5))
    c = [0.6, 0.5, 0.4, 0.3, 0.25][severity - 1]  # 對應縮放比例
    
    # 進行 pixelate 處理
    x = x.resize((int(224 * c), int(224 * c)), PILImage.LANCZOS)  # 使用 LANCZOS 提高效果
    x = x.resize((224, 224), PILImage.LANCZOS)

    # 轉回 NumPy 陣列
    x = np.array(x)

    return 255 - x


# mod of https://gist.github.com/erniejunior/601cdf56d2b424757de5
def elastic_transform(image, severity=1):
    c = [(244 * 2, 244 * 0.7, 244 * 0.1),   # 244 should have been 224, but ultimately nothing is incorrect
         (244 * 2, 244 * 0.08, 244 * 0.2),
         (244 * 0.05, 244 * 0.01, 244 * 0.02),
         (244 * 0.07, 244 * 0.01, 244 * 0.02),
         (244 * 0.12, 244 * 0.01, 244 * 0.02)][severity - 1]

    image = np.array(image, dtype=np.float32) / 255.
    shape = image.shape
    shape_size = shape[:2]

    # random affine
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size,
                       [center_square[0] + square_size, center_square[1] - square_size],
                       center_square - square_size])
    pts2 = pts1 + np.random.uniform(-c[2], c[2], size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)
    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)

    dx = (gaussian(np.random.uniform(-1, 1, size=shape[:2]),
                   c[1], mode='reflect', truncate=3) * c[0]).astype(np.float32)
    dy = (gaussian(np.random.uniform(-1, 1, size=shape[:2]),
                   c[1], mode='reflect', truncate=3) * c[0]).astype(np.float32)
    dx, dy = dx[..., np.newaxis], dy[..., np.newaxis]

    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
    indices = np.reshape(y + dy, (-1, 1)), np.reshape(x + dx, (-1, 1)), np.reshape(z, (-1, 1))
    return np.clip(map_coordinates(image, indices, order=1, mode='reflect').reshape(shape), 0, 1) * 255


In [ ]:
##################################################################################################################################
def reverse_normalize(image, mean, std):
    # 確保均值和標準差是 numpy array，以支持逐像素計算
    mean = np.array(mean).reshape(1, 1, -1)  # [1, 1, C]
    std = np.array(std).reshape(1, 1, -1)    # [1, 1, C]

    # 反標準化公式
    image = (image * std) + mean
    return np.clip(image, 0, 255)  # 確保範圍在 [0, 255]

def normalize(image, mean, std):
    # 將影像標準化
    mean = np.array(mean).reshape(1, 1, -1)  # [1, 1, C]
    std = np.array(std).reshape(1, 1, -1)    # [1, 1, C]
    image = (image - mean) / std
    return image

# Custom dataset to apply elastic_transform
# 自定義數據集
class IndexBasedTransformDataset(Dataset):
    def __init__(self, original_dataset, transforms_by_range, repeat_factor=1):
        """
        original_dataset: 原始 PyTorch Dataset
        transforms_by_range: 列表，每個元素是 (start_index, end_index, transform_function)
        repeat_factor: 數據集重複的倍數
        """
        self.original_dataset = original_dataset
        self.transforms_by_range = transforms_by_range
        self.repeat_factor = repeat_factor  # 數據集重複幾次

    def __len__(self):
        return len(self.original_dataset) * self.repeat_factor

    def __getitem__(self, idx):
        # 對數據索引進行取模，將重複部分對應回原始數據
        original_idx = idx % len(self.original_dataset)
        image, label = self.original_dataset[original_idx]
        
        # 轉換影像形狀為 [H, W, C]
        if isinstance(image, torch.Tensor):
            image = image.permute(1, 2, 0).numpy()  # 從 [C, H, W] 變為 [H, W, C]
        
        # Step 1: 反標準化
        image = reverse_normalize(image, mean=[103.939, 116.779, 123.68], std=[1.0, 1.0, 1.0])

        # Step 2: 影像增強
        for start, end, transform in self.transforms_by_range:
            if start <= idx < end:
                if transform is not None:
                    image = transform(image)
                break
        
        # Step 3: 標準化
        image = normalize(image, mean=[103.939, 116.779, 123.68], std=[1.0, 1.0, 1.0])

        # 還原影像形狀為 [C, H, W]
        image = np.transpose(image, (2, 0, 1))  # 從 [H, W, C] 變回 [C, H, W]
        return image, label
    
test_dataset, _ = random_split(dataset, [limit, len(dataset) - limit])
image, _ = test_dataset[0]
print(image.shape)
'''corruption_types = ['brightness', 'contrast', 'defocus_blur', 'elastic_transform', 'fog', 
                    'frost', 'gaussian_blur', 'gaussian_noise', 'glass_blur', 'impulse_noise', 
                    'pixelate', 'saturate', 'shot_noise', 'spatter', 'speckle_noise', 'zoom_blur']'''

severity = 5
# 為每 1000 筆設置對應的增強方法
transforms_by_range = [
    (0, limit, lambda img: brightness(img, severity=severity)),
    (limit, 2 * limit, lambda img: contrast(img, severity=severity)),
    (2 * limit, 3 * limit, lambda img: defocus_blur(img, severity=severity)),
    (3 * limit, 4 * limit, lambda img: elastic_transform(img, severity=severity)),
    (4 * limit, 5 * limit, lambda img: fog(img, severity=severity)),
    (5 * limit, 6 * limit, lambda img: gaussian_blur(img, severity=severity)),
    (6 * limit, 7 * limit, lambda img: gaussian_noise(img, severity=severity)),
    (7 * limit, 8 * limit, lambda img: glass_blur(img, severity=severity)),
    (8 * limit, 9 * limit, lambda img: impulse_noise(img, severity=severity)),
    (9 * limit, 10 * limit, lambda img: pixelate(img, severity=severity)), ## need check
    (10 * limit, 11 * limit, lambda img: saturate(img, severity=severity)),
    (11 * limit, 12 * limit, lambda img: shot_noise(img, severity=severity)),
    (12 * limit, 13 * limit, lambda img: spatter(img, severity=severity)),
    (13 * limit, 14 * limit, lambda img: speckle_noise(img, severity=severity)),
    (14 * limit, 15 * limit, lambda img: zoom_blur(img, severity=severity)),
    #(15 * limit, 16 * limit, None),
]

test_dataset = IndexBasedTransformDataset(test_dataset, transforms_by_range, len(transforms_by_range))

# 創建 DataLoader
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=6, pin_memory=True)

In [ ]:
conf_vanilla = []
# 預測並顯示分數

correct = 0
total = 0
img, _ = test_dataset[0]
print(len(test_dataset))


'''fig, axes = plt.subplots(3, 5, figsize=(15, 9))
class_names = {0:'brightness', 1:'contrast', 2:'defocus_blur', 3:'elastic_transform', 4:'fog', 
                    5:'gaussian_blur', 6:'gaussian_noise', 7:'glass_blur', 8:'impulse_noise', 
                    9:'pixelate', 10:'saturate', 11:'shot_noise', 12:'spatter', 13:'speckle_noise', 14:'zoom_blur'}  # 你的類別名稱
for i in range(15):
    index = limit * i  # 計算索引
    img, _ = test_dataset[index]  # 讀取影像

    # 反正規化影像
    img = reverse_normalize(img.transpose(1, 2, 0), mean=[103.939, 116.779, 123.68], std=[1.0, 1.0, 1.0]) / 255
    
    ax = axes[i // 5, i % 5]  # 計算子圖位置
    ax.imshow(img)
    ax.set_title(f"{class_names[i]}", fontsize=12)
    ax.axis("off")  # 隱藏座標軸

plt.tight_layout()
plt.show()'''

In [ ]:
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device).float()
        labels = labels.to(device)
        outputs = model(images)

        # 計算Softmax並獲取最大值
        softmax_probs = F.softmax(outputs, dim=1)  # Softmax值
        max_probs, predicted = torch.max(softmax_probs, 1)

        conf_vanilla.extend(max_probs.cpu().numpy()) 
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #print(model)
        #print(labels, predicted)
        
conf_vanilla = np.array(conf_vanilla)
accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
# bzq modifying
len_x = len_x_target
len_y = len_y_target
stride_x = stride_x_target
stride_y = stride_y_target
batch_size = 1  # 設定批次大小

original_predictions_imagenet = []

with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        batch_data = batch_data.to(device).float()
        batch_predictions_imagenet = model(batch_data).cpu().numpy()
        original_predictions_imagenet.append(batch_predictions_imagenet)

original_predictions_imagenet = np.vstack(original_predictions_imagenet)

# 使用正確的方式來訪問 test_dataset 中的標籤
test_labels = [test_dataset[i][1] for i in range(len(test_dataset))]

predicted_labels = np.argmax(original_predictions_imagenet, axis=1)
correct_indices = np.where(predicted_labels == np.array(test_labels))[0]
incorrect_indices = np.where(predicted_labels != np.array(test_labels))[0]

correct_predictions_imagenet.extend(correct_indices.tolist())
incorrect_predictions_imagenet.extend(incorrect_indices.tolist())

print(f"{len(correct_predictions_imagenet)}, {len(incorrect_predictions_imagenet)}")

'''original_predictions_imagenet = []
with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        batch_predictions_imagenet = model(batch_data)
        original_predictions_imagenet.append(batch_predictions_imagenet.cpu().numpy())

for i in range(len(test_dataset)):
    if np.argmax(original_predictions_imagenet[i]) == test_dataset[i][1]:
        correct_predictions_imagenet.append(i)
    else:
        incorrect_predictions_imagenet.append(i)

original_predictions_imagenet = np.vstack(original_predictions_imagenet)

print(f"{len(correct_predictions_imagenet)}, {len(incorrect_predictions_imagenet)}")'''


In [ ]:
import torch
import numpy as np
from collections import Counter
import torch.nn.functional as F

random_num_for_bzq_mask_imagenet = np.random.randint(0, 256, (3, len_y_target, len_x_target)).astype(np.float32) / 255.0

def brier_score_decomposition(predictions, labels, num_classes=365):
    """
    計算 Brier Score 並拆解為 Uncertainty, Resolution, Reliability
    predictions: (batch_size, num_classes) - softmax 輸出
    labels: (batch_size) - 正確標籤
    """
    labels_one_hot = torch.nn.functional.one_hot(labels, num_classes).float()

    # 計算 Brier Score
    brier_score = torch.mean(torch.sum((predictions - labels_one_hot) ** 2, dim=1))

    # 計算 Uncertainty
    marginal_probs = torch.mean(predictions, dim=0)
    uncertainty = torch.sum(marginal_probs * (1 - marginal_probs))

    # 計算 Resolution
    mean_class_probs = torch.mean(labels_one_hot, dim=0)
    resolution = torch.sum(mean_class_probs * (1 - mean_class_probs))

    # 計算 Reliability
    reliability = brier_score - uncertainty + resolution  # 修正計算方式

    return {
        "Brier Score": brier_score.item(),
        "Uncertainty": uncertainty.item(),
        "Resolution": resolution.item(),
        "Reliability": reliability.item()
    }

def compute_nll(predictions, labels):
    """
    計算 Negative Log-Likelihood (NLL)
    predictions: shape (batch_size, num_classes) - softmax 輸出
    labels: shape (batch_size) - 正確的標籤
    """
    log_probs = torch.log(torch.clamp(predictions, min=1e-9))  # 避免 log(0) 問題
    labels = labels.expand(predictions.size(0))
    nll_loss = F.nll_loss(log_probs, labels)
    return nll_loss.item()

def process_batch(batch_data, batch_labels, len_y, stride_y, len_x, stride_x, device, model, alpha, bzq):
    if len(batch_data) == 0:
        return bzq, np.array([])

    targets = []

    for i in range(0, 224 - len_y, stride_y):
        for j in range(0, 224 - len_x, stride_x):
            #target = single_data_bzq_mask_preprocessing_imagenet(batch_data, i, j, len_x, len_y, 0)
            target = single_data_bzq_mask_preprocessing_imagenet_random_global(batch_data, i, j, len_x, len_y, random_num_for_bzq_mask_imagenet)
            targets.append(target)
    
    targets_tensor = torch.from_numpy(np.vstack(targets).reshape(-1, 3, 224, 224)).float().to(device)
    predictions = model(targets_tensor)
    
    #temp, _ = torch.max(F.softmax(predictions), dim=1)
    #temp = temp.cpu().numpy()
    
    max_bzq_indices = torch.argmax(predictions, dim=1).cpu().numpy()
    
    # 計算 softmax
    softmax_predictions = F.softmax(predictions, dim=1)

    labels = batch_labels

    # 計算 Brier Score 分解與 NLL
    brier_components = brier_score_decomposition(softmax_predictions, labels)
    nll_loss = compute_nll(softmax_predictions, labels)

    counter = Counter(max_bzq_indices)
    most_common_num, most_common_count = counter.most_common(1)[0]
    
    temp = softmax_predictions[:, most_common_num].cpu().numpy()
    
    bzq.append(alpha * np.mean(temp) + (1 - alpha) * (2.0 / np.pi * np.arctan(1.0 / np.std(temp))))

    return bzq, temp, brier_components, nll_loss

# 初始化儲存 Brier Score 分解部分與 NLL 的陣列
brier_scores = []
uncertainties = []
resolutions = []
reliabilities = []
nll_losses = []

with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        batch_data = batch_data.to(device)  # 將批次影像搬到指定裝置
        batch_labels = batch_labels.to(device)
        if len(batch_data) > 0:
            bzq, temp, brier_components, nll_loss = process_batch(
                batch_data, batch_labels, len_y, stride_y, len_x, stride_x, device, model, alpha, bzq
            )

            brier_scores.append(brier_components["Brier Score"])
            uncertainties.append(brier_components["Uncertainty"])
            resolutions.append(brier_components["Resolution"])
            reliabilities.append(brier_components["Reliability"])
            nll_losses.append(nll_loss)

            # 影像預處理
            original_data = single_data_bzq_mask_preprocessing_imagenet(batch_data, 0, 0, 0, 0, 0)
            original_prediction = model(torch.tensor(original_data.reshape(-1, 3, 224, 224)).float().to(device))
            max_original_index = torch.argmax(original_prediction).item()

            # 根據 `bzq` 值分類統計
            if bzq[-1] == 0.0:
                if (len(bzq) - 1) in correct_predictions_imagenet:
                    correct_predictions_bzq_zero_softmax_mean.append(np.mean(temp))
                    correct_predictions_bzq_zero_softmax_std.append(np.std(temp))
                else:
                    incorrect_predictions_bzq_zero_softmax_mean.append(np.mean(temp))
                    incorrect_predictions_bzq_zero_softmax_std.append(np.std(temp))

bzq_imagenet = np.array(bzq)



In [ ]:
#result_bzq_imagenet = 1 / bzq_imagenet
result_bzq_imagenet = bzq_imagenet        


counts, bins, patches = plt.hist(bzq_imagenet, bins=bins_size)
'''plt.title('Cumulative Histogram of Correct Predictions')
plt.xlabel('bzq')
plt.ylabel('Count')
plt.legend(loc='upper right')  # 指定圖例位置
plt.show()

# 打印結果
plt.boxplot(bzq_imagenet)
plt.show()'''

bzq_correct_imagenet = np.array([bzq_imagenet[i] for i in correct_predictions_imagenet])
bzq_incorrect_imagenet = np.array([bzq_imagenet[i] for i in incorrect_predictions_imagenet])

result_bzq_correct_imagenet = np.array([result_bzq_imagenet[i] for i in correct_predictions_imagenet])
result_bzq_incorrect_imagenet = np.array([result_bzq_imagenet[i] for i in incorrect_predictions_imagenet])

# 繪製分布圖
plt.figure(figsize=(5, 3))
sns.histplot(correct_predictions_bzq_zero_softmax_mean, bins=15) 
plt.show()
# 繪製分布圖
plt.figure(figsize=(5, 3))
sns.histplot(incorrect_predictions_bzq_zero_softmax_mean, bins=15)
plt.show()
# 繪製分布圖
plt.figure(figsize=(5, 3))
sns.histplot(correct_predictions_bzq_zero_softmax_std, bins=15) 
plt.show()
# 繪製分布圖
plt.figure(figsize=(5, 3))
sns.histplot(incorrect_predictions_bzq_zero_softmax_std, bins=15) 
plt.show()

In [ ]:

# Separate correct and incorrect predictions
bzq_correct_imagenet = np.array([bzq_imagenet[i] for i in correct_predictions_imagenet])
bzq_incorrect_imagenet = np.array([bzq_imagenet[i] for i in incorrect_predictions_imagenet])

result_bzq_correct_imagenet = np.array([result_bzq_imagenet[i] for i in correct_predictions_imagenet])
result_bzq_incorrect_imagenet = np.array([result_bzq_imagenet[i] for i in incorrect_predictions_imagenet])

# Create a figure with subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

# Cumulative Histogram of Correct Predictions
axs[0, 0].hist(bzq_correct_imagenet, bins=bins_size)
axs[0, 0].set_title('Cumulative Histogram of Correct Predictions')
axs[0, 0].set_xlabel('bzq')
axs[0, 0].set_ylabel('Count')
axs[0, 0].legend(['Correct Predictions'], loc='upper right')

# Boxplot of bzq_imagenet
axs[0, 1].boxplot(bzq_correct_imagenet)
axs[0, 1].set_title('Boxplot of bzq_imagenet')

# Cumulative Histogram of Correct Predictions
axs[1, 0].hist(bzq_incorrect_imagenet, bins=bins_size)
axs[1, 0].set_title('Cumulative Histogram of Incorrect Predictions')
axs[1, 0].set_xlabel('bzq')
axs[1, 0].set_ylabel('Count')
axs[1, 0].legend(['Incorrect Predictions'], loc='upper right')

# Boxplot of bzq_correct_imagenet
axs[1, 1].boxplot(bzq_incorrect_imagenet)
axs[1, 1].set_title('Boxplot of bzq_incorrect_imagenet')

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
# 劃出confidence-acc 圖: confidence由bzq提供，acc由該confidence數值底下預測準確的

result_pred_imagenet = np.ones(len(test_dataset)) 
for i in incorrect_predictions_imagenet:
    result_pred_imagenet[i] = 0

print(sum(result_pred_imagenet))

result_imagenet_dict = {}
for i, val in enumerate(result_bzq_imagenet):
    if val not in result_imagenet_dict.keys():
        result_imagenet_dict[val] = [result_pred_imagenet[i]]
    else:
        result_imagenet_dict[val].append(result_pred_imagenet[i])

# 初始化信心值和準確率列表
confidence_values = []
accuracies = []
element_counts = []

# 計算每個信心值範圍的準確率
for confidence in sorted(result_imagenet_dict.keys(), reverse=True):
    combined_results = []
    for key in result_imagenet_dict:
        if key >= confidence:
            combined_results.extend(result_imagenet_dict[key])
    element_count = len(combined_results)
    accuracy = np.mean(combined_results)
    confidence_values.append(confidence)
    accuracies.append(accuracy)
    element_counts.append(element_count)

# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values, accuracies, marker='o', linestyle='-', color='b')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Accuracy (p(y|x) >= τ)')
plt.title('Confidence vs Accuracy (Rotated 60°)')
plt.grid(True)
plt.show()

# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values, element_counts, marker='o', linestyle='-', color='b')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Number of Elements (p(y|x) >= τ)')
plt.title('Confidence Threshold vs Number of Elements')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

confidence_values_scaled = np.array(confidence_values)
#confidence_values_scaled = 2 / np.pi * np.arctan(confidence_values_scaled)
#confidence_values_scaled = confidence_values_scaled * confidence_values_scaled / (1 - confidence_values_scaled * confidence_values_scaled)
                                                                                  

#print(confidence_values_scaled)

# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values_scaled, accuracies, marker='o', linestyle='-', color='b')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Accuracy (p(y|x) >= τ)')
plt.title('Confidence vs Accuracy (Rotated 60°)')
plt.grid(True)
plt.show()

scaler = MinMaxScaler()
confidence_values_scaled = scaler.fit_transform(np.array(confidence_values_scaled).reshape(-1, 1)).flatten()
#print(confidence_values_scaled)

# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values_scaled, accuracies, marker='o', linestyle='-', color='b')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Accuracy (p(y|x) >= τ)')
plt.title('Confidence vs Accuracy ')
plt.grid(True)
plt.show()

# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values_scaled, element_counts, marker='o', linestyle='-', color='b')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Accuracy (p(y|x) >= τ)')
plt.title('Confidence vs Accuracy')
plt.grid(True)
plt.show()

In [ ]:
from collections import defaultdict
#vanilla
#original_predictions_imagenet (800000, 10)
# 初始化信心值和準確率列表

# 初始化 confidence_map_vanilla 為 defaultdict
confidence_map_vanilla = defaultdict(list)

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

# 將預測結果和信心值存入字典
for i, val in enumerate(original_predictions_imagenet):
    conf = np.max(softmax(val))
    confidence_map_vanilla[conf].append(result_pred_imagenet[i])

print("finish")
print(len(confidence_map_vanilla))

confidence_values_vanilla = []
accuracies_vanilla = []
element_counts_vanilla = []

# 計算每個信心值範圍的準確率
sorted_confidences = sorted(confidence_map_vanilla.keys(), reverse=True)
combined_results_vanilla = []

for confidence in sorted_confidences:
    combined_results_vanilla.extend(confidence_map_vanilla[confidence])
    element_count_vanilla = len(combined_results_vanilla)
    accuracy_vanilla = np.mean(combined_results_vanilla)
    confidence_values_vanilla.append(confidence)
    accuracies_vanilla.append(accuracy_vanilla)
    element_counts_vanilla.append(element_count_vanilla)
 


In [ ]:
'''
# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values_vanilla, accuracies_vanilla, marker='o', linestyle='-', color='b')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Accuracy (p(y|x) >= τ)')
plt.title('Confidence vs Accuracy')
plt.grid(True)
plt.show()'''

# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values_scaled, element_counts, marker='.', linestyle='-', label='Scaled', markersize=4)
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Number of Elements (p(y|x) >= τ)')
plt.title('Confidence Threshold vs Number of Elements')
plt.legend()
plt.grid(True)
plt.show()

'''
# 繪製圖形
plt.figure(figsize=(10, 6))
plt.plot(confidence_values_vanilla, accuracies_vanilla, marker='o', linestyle='-', color='b', label='Vanilla')
plt.plot(confidence_values_scaled, accuracies, marker='o', linestyle='-', color='r', label='Scaled')
plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Accuracy (p(y|x) >= τ)')
plt.title('Confidence vs Accuracy')
plt.legend()
plt.grid(True)
plt.show()'''

plt.figure(figsize=(10, 6))
plt.plot(confidence_values_scaled, accuracies, marker='.', linestyle='-', label='Scaled', markersize=4)

# 新增垂直線
plt.axvline(x=0.6827, color='g', linestyle='--', label='x=0.6827')
plt.axvline(x=0.9545, color='m', linestyle='--', label='x=0.9545')
plt.axvline(x=0.9973, color='c', linestyle='--', label='x=0.9973')

# 找到最接近的值
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

idx_6827_scaled = find_nearest(confidence_values_scaled, 0.6827)
idx_9545_scaled = find_nearest(confidence_values_scaled, 0.9545)
idx_9973_scaled = find_nearest(confidence_values_scaled, 0.9973)


plt.scatter([confidence_values_scaled[idx_6827_scaled], confidence_values_scaled[idx_9545_scaled], confidence_values_scaled[idx_9973_scaled]], 
            [accuracies[idx_6827_scaled], accuracies[idx_9545_scaled], accuracies[idx_9973_scaled]], 
            zorder=5)

plt.text(confidence_values_scaled[idx_6827_scaled], accuracies[idx_6827_scaled], f'({confidence_values_scaled[idx_6827_scaled]:.3f}, {accuracies[idx_6827_scaled]:.3f})', fontsize=14, ha='right') 
plt.text(confidence_values_scaled[idx_9545_scaled], accuracies[idx_9545_scaled], f'({confidence_values_scaled[idx_9545_scaled]:.3f}, {accuracies[idx_9545_scaled]:.3f})', fontsize=14, ha='right') 
plt.text(confidence_values_scaled[idx_9973_scaled], accuracies[idx_9973_scaled], f'({confidence_values_scaled[idx_9973_scaled]:.3f}, {accuracies[idx_9973_scaled]:.3f})', fontsize=14, ha='right')

plt.xlabel('Confidence Threshold (τ)')
plt.ylabel('Accuracy (p(y|x) >= τ)')
plt.title('Confidence vs Accuracy')
plt.legend()
plt.grid(True)
plt.show()

print(accuracies[idx_6827_scaled], accuracies[idx_9545_scaled], accuracies[idx_9973_scaled])


In [ ]:
# 存儲到 .npy 檔案 
print(random_num_for_bzq_mask_imagenet)
'''np.save('confidence_values_vanilla.npy', confidence_values_vanilla) 
np.save('accuracies_vanilla.npy', accuracies_vanilla) 
np.save('element_counts_vanilla.npy', element_counts_vanilla)'''

In [ ]:
result_bzq_imagenet_modified = scaler.fit_transform(np.array(result_bzq_imagenet).reshape(-1, 1)).flatten()
result_bzq_imagenet_modified = result_bzq_imagenet
print(np.sum([item for sublist in confidence_map_vanilla.values() for item in sublist]))
#ECE calc

def calculate_ece(confidences, labels, num_bins=15):
    bin_boundaries = np.linspace(0, 1, num_bins + 1)
    bin_lowers = bin_boundaries[:-1]
    bin_uppers = bin_boundaries[1:]

    ece = 0.0
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        in_bin = np.logical_and(confidences > bin_lower, confidences <= bin_upper)
        prop_in_bin = np.mean(in_bin)
        if prop_in_bin > 0:
            accuracy_in_bin = np.mean(labels[in_bin])
            avg_confidence_in_bin = np.mean(confidences[in_bin])
            ece += np.abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

    return ece


print(result_bzq_imagenet)
# 計算ECE
ece = [calculate_ece(result_bzq_imagenet_modified[limit * i: limit * (i + 1)], result_pred_imagenet[limit * i: limit * (i + 1)]) for i in range(15)]

ece_vanilla = [calculate_ece(conf_vanilla[limit * i: limit * (i + 1)], result_pred_imagenet[limit * i: limit * (i + 1)]) for i in range(15)]

print("Expected Calibration Error (ECE):", ece)
fig, ax = plt.subplots() 
ax.boxplot(ece) 
ax.set_title('ECE Boxplot') 
ax.set_ybound(0, 0.7)
ax.set_ylabel('ECE') 
plt.show()

print("Expected Calibration Error (ECE):", ece_vanilla)
fig, ax = plt.subplots() 
ax.boxplot(ece_vanilla) 
ax.set_title('ECE Boxplot') 
ax.set_ybound(0, 0.7)
ax.set_ylabel('ECE') 
plt.show()
print(ece, ece_vanilla)
print(np.percentile(ece, 25), np.percentile(ece, 50), np.percentile(ece, 75))
print(np.percentile(ece_vanilla, 25), np.percentile(ece_vanilla, 50), np.percentile(ece_vanilla, 75))


In [ ]:
#print(ece_modified)
ece = [calculate_ece(result_bzq_imagenet_modified[limit * i: limit * (i + 1)], result_pred_imagenet[limit * i: limit * (i + 1)]) for i in range(15)]
nll = [np.mean(nll_losses[limit * i : limit * (i + 1)]) for i in range(len(nll_losses) // limit)]
bs = [np.mean(brier_scores[limit * i : limit * (i + 1)]) for i in range(len(brier_scores) // limit)]
top5_ece = sorted(ece, reverse=False)[:5]
print(ece)
print(nll)
print(bs)
plt.figure(figsize=(10, 5))
plt.boxplot(ece)
plt.ylim(0, 1)
plt.title("ECE")
plt.legend()
plt.show()
plt.figure(figsize=(10, 5))
plt.boxplot(top5_ece)
plt.ylim(0, 1)
plt.title("ECE top 5")
plt.legend()
plt.show()
plt.figure(figsize=(10, 5))
plt.boxplot(nll)
plt.ylim(0, 12)
plt.title("NLL")
plt.legend()
plt.show()
plt.figure(figsize=(10, 5))
plt.boxplot(bs)
plt.ylim(0, 1.6)
plt.title("Brier Score")
plt.legend()
plt.show()
